# Algolia - Alternative searches

The provided dataset contains one month of anonymized aggregated searches coming from HN Search. In this case study, we want to provide alternative queries according to a user query.

We propose to use a item based collaborative filtering approach in order to suggest queries related to another query.

###### From logs to usable data
First, for each document we want it to be described by a set of related queries. To do that, we will use the query and the hits (top 10 results according to the query) fields of the search json line.
Each time a record is in the hits, we will add the query as a description of the record. We will obtain the  vector format:
ObjectID : [q0, q1, ... qn]
where 
    ObjectID the internal identifier of the record 
    [q0, q1, ..., qn] the set of search queries describing the record
Finally, we will store these vectors in an Elasticsearch indice.

##### Get recommendations as alternative searches
Elasticsearch aggregations are commonly used for faceted search feature. But Significant Terms Aggregation measures the kind of statistically significant relationships to deliver meaningful recommendations. It calculates the statistical significance of terms in the current results when compared to the background corpus.
Applied to our use case, it means that for a user query, we will retrieve all records containing this query, and then calculate the significance of the queries in the current record results and provide the most signifcant queries to the user query.




In [100]:
from os import listdir
from os.path import isfile, join
import json
import string
from elasticsearch import Elasticsearch, helpers

We make the following assumption to keep a query as a candidate for an alternative search:

the query must not be empty
the query must be equal or more than 3 chars
the query must provide search results (in order to not suggest queries that retrieve no result to the user)

In [75]:
def is_valid_query(data):
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    query = data['query'].translate(translator)
    if not query or len(query) < 3 or data['nb_hits'] <= 0:
        return False
    return True

def clean_query(query):
        translator = str.maketrans('', '', string.punctuation)
        query = data['query'].translate(translator)
        return query.lower().strip()

def levenshtein(s, t):
        ''' From Wikipedia article; Iterative with two matrix rows. '''
        if s == t: return 0
        elif len(s) == 0: return len(t)
        elif len(t) == 0: return len(s)
        v0 = [None] * (len(t) + 1)
        v1 = [None] * (len(t) + 1)
        for i in range(len(v0)):
            v0[i] = i
        for i in range(len(s)):
            v1[0] = i + 1
            for j in range(len(t)):
                cost = 0 if s[i] == t[j] else 1
                v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
            for j in range(len(v0)):
                v0[j] = v1[j]
                
        return v1[len(t)]

##### Create query vector for each record in the dataset
ObjectID : [q0, q1, ... qn] 

In [25]:
path = "workspace/algolia/hn_insights_data/"
files = [f for f in listdir(path) if isfile(join(path, f))]
files.sort()

doc = {}
for f in files:
    with open(path + f, 'r') as logs:
         for line in logs.readlines():
                try:
                    # load json data
                    data = json.loads(line)
                    if is_valid_query(data):
                        if 'hits' in data:
                            for h in data['hits']:
                                if h not in doc.keys():
                                    doc[h] = []
                                query = clean_query(data['query'])
                                if not query in doc[h]:
                                    doc[h].append(query)
                                
                except ValueError:
                    ## Line is not a valid JSON, we skip this line
                    ## print("JSON decode as failed for line: %s" %line)
                    continue



###### Display some examples of a query vector

In [27]:
n = 0
for d in doc:
    print('%s : %s ' %(d, doc[d]))
    n += 1
    if n > 10:
        break

14158772 : ['real estate', 'estate', 'beyond burger', 'make money', 'mcdonalds', 'est', 'real es', 'estb', 'real estrate t'] 
4638286 : ['real estate', 'estate', 'zynga', 'plum', 'real es', 'plumm', 'plummer', 'plumme', 'plummers', 'below'] 
14719749 : ['real estate', 'ipo', 'estate', 'redfin', 'files for ipo', 'real es'] 
15508956 : ['real estate', 'wework', 'estate', 'wwork', 'critics say wework is an overvalued realestate play', 'real es', 'we work', 'overvalued'] 
10728058 : ['real estate', 'estate', 'real es'] 
1646087 : ['real estate', 'estate', 'jason fried', 'real es'] 
10263075 : ['real estate', 'oakland', 'oak', 'lease', 'real es'] 
14874910 : ['real estate', 'redfin', 'real es', 'shares s', 'real estrate t', 'redfin shares surge more than 30 in 1385m real estate tech ipo'] 
12160680 : ['real estate', 'columbia', 'columbii', 'real es'] 
15750785 : ['real estate', 'real es'] 
3374434 : ['bird', 'angry', 'birh', 'bir', 'create', 'inverse', 'show hn', 'inver', 'angr', 'ang', 'in

###### Create an elasticsearch indice with a specific mapping
The field id corresponds to the record ObjectId and the queries field corresponds to the set of queries describing a record.

In [38]:
es = Elasticsearch(['http://localhost:9200'], timeout=14400, http_compress=True, max_retries=3)
mapping = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "docs": {
            "properties": {
                "queries": {
                    "type": "keyword"
                },
                "id": {
                    "type": "long"
                }
            }
        }
     }
}

es.indices.create(index='alternatives', ignore=400, body=mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'alternatives'}

##### Indexing data in the Elasticsearch indice
We use the parallel bulk Elasticsearch api in order to speed up indexing.

In [39]:
def generate_actions(docs):
    for d in docs:
        json_data = {}
        json_data["id"] = d
        json_data["queries"] = doc[d]
        yield {
            '_op_type': 'index',
            '_index': 'alternatives',
            '_type': 'docs',
            '_id': d,
            '_source': json_data
        }
total = 0
bulks = 0
for success, result in helpers.parallel_bulk(
    client = es,actions=generate_actions(doc),
    chunk_size=5000,
    thread_count=4,
    raise_on_error=False,
    raise_on_exception=False
):
    total += 1
    if total % 5000 == 0:
        bulks += 1
        print('Bulk index #%s - Total documents: %s' % (bulks, total))

Bulk index #1 - Total documents: 5000
Bulk index #2 - Total documents: 10000
Bulk index #3 - Total documents: 15000
Bulk index #4 - Total documents: 20000
Bulk index #5 - Total documents: 25000
Bulk index #6 - Total documents: 30000
Bulk index #7 - Total documents: 35000
Bulk index #8 - Total documents: 40000
Bulk index #9 - Total documents: 45000
Bulk index #10 - Total documents: 50000
Bulk index #11 - Total documents: 55000
Bulk index #12 - Total documents: 60000
Bulk index #13 - Total documents: 65000
Bulk index #14 - Total documents: 70000
Bulk index #15 - Total documents: 75000
Bulk index #16 - Total documents: 80000
Bulk index #17 - Total documents: 85000
Bulk index #18 - Total documents: 90000
Bulk index #19 - Total documents: 95000
Bulk index #20 - Total documents: 100000
Bulk index #21 - Total documents: 105000
Bulk index #22 - Total documents: 110000
Bulk index #23 - Total documents: 115000
Bulk index #24 - Total documents: 120000
Bulk index #25 - Total documents: 125000
Bulk

###### Get the recommendations
Build the significant terms aggregation query in order to get alternative searches of a query. We are using the mutual information as significance score in order to boost common queries. The score could be change according to the use case for one which is a compromise between common and rare (JLH) or one which foster rare queries (google normalized distance)

In [ ]:
def build_aggregation_query(query):
    return {
        "query": {
            "match": {"queries": query}
        },
        "aggs": {
            "significant_queries": {
            "significant_terms": {"field": "queries", "mutual_information":{}, "size": 50}
            }
        },
        "size": 0
    }

###### Display several examples of alternative searches

In [99]:
test_queries = ['macbook', 'python', 'tesla', 'russia', 'trump', 'matrix riot', 'information retrieval', 'hacker', 'money']
for q in test_queries:
    print('Alternative searches for "%s" query: ' %q)
    res = es.search(index='alternatives', body=build_aggregation_query(q))
    n = 0
    for bucket in res['aggregations']['significant_queries']['buckets']:
        if (bucket['key']) == q or (bucket['key'].startswith(q) and len(bucket['key'].split(' ')) == len(q.split(' '))) or levenshtein(bucket['key'], q) <= 3 :
            continue
        print('%s | %f' %(bucket['key'], bucket['score']))
        n += 1
        if n > 10:
            break
    print("--------------------------------")

Alternative searches for "macbook" query: 
macbook pro | 0.001053
mac book pro | 0.000438
mac | 0.000326
macbook pro 2018 | 0.000324
keyboard | 0.000323
apple macbook | 0.000283
apple | 0.000276
macbook pro keyboard | 0.000263
macbook keyboard | 0.000220
new macbook | 0.000199
macbook pro keybaord | 0.000178
--------------------------------
Alternative searches for "python" query: 
ruby | 0.000210
rust | 0.000127
java | 0.000122
perl | 0.000115
python type | 0.000112
github | 0.000110
python data | 0.000100
javascript | 0.000095
python web | 0.000094
python google | 0.000093
python machine learning | 0.000087
--------------------------------
Alternative searches for "tesla" query: 
elon musk | 0.000487
model 3 | 0.000477
elon | 0.000467
musk | 0.000463
tesla autopilot | 0.000187
autopilot | 0.000163
tesla model 3 | 0.000149
tesla whistleblower | 0.000124
spacex | 0.000121
model m | 0.000116
tesla sabotage | 0.000111
--------------------------------
Alternative searches for "russia" que

##### Conclusion
Using item based collaborative filtering approach gives good alternative searches. Moreover, with Elasticsearch and the significant terms aggregation, it is relatively easy to implement.
However, there are several limitations to this approach. First, there is a need of a lot of data to build relevant relationships between items. Secondly, we can see with the examples in this notebook that a pre processing of the data and post processing of the results would be essential to provide relevant alternative queries. As we can notice, there is a lot of queries that have the same meaning and also queries that are not understandable for users (incomplete words, misspelled words, ...)